ipynb파일 실행

In [3]:
!pip install import_ipynb
import import_ipynb

In [12]:
import dataset

In [17]:
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader
from model import Yolov1
from datasets import VOCDataset
from util import ( 
    non_max_suppression,
    mean_average_precision,
    intersection_over_union, #IoU 불러와야할듯
    cellboxes_to_boxes, #convert from cell relative to entire image
    get_bboxes,#convert and get all bbx
    plot_image,
    save_checkpoint,
    load_checkpoint,
)
#백엔드 할당??? 

importing Jupyter notebook from datasets.ipynb
importing Jupyter notebook from util.ipynb


error feedback: utils로 했을때 importerror가 나서 util로 ipynb파일명 변경후 from util 불러옴
임포트 에러날때- ipynb파일명 바꿈

In [18]:
from loss import YoloLoss

seed = 123
torch.manual_seed(seed) #get same data set loading

# Hyperparameters etc. 
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 16 # 64 in original paper but I don't have that much vram, grad accum?
#batch size 64라고 언급된 논문 본적 없음, gpu 업그레이드 필요함
WEIGHT_DECAY = 0 #not train entire model- take a long time to train , 계산상의 이유로 무거운 데이터 증강, computational reasons
#정규화 없음
EPOCHS = 1000
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar" #오버피팅하고 나중에 실행할 예정
IMG_DIR = "C:/Users/Owner/Downloads/archive/images"
LABEL_DIR = "C:/Users/Owner/Downloads/archive/labels"

#resize, transform.compose 는 bbx 포함 안할 수도
#resize 하면 조금 이상하게 보일 수도? not do any data augmentation on the label side
#데이터 증강이 많았으면 데이터 확장 쉬웠을것이다.
class Compose(object): 
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes = t(img), bboxes #t- transform, 이미지를 작동시키기 위함, 크기 조정만 할것임

        return img, bboxes


transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])
#resize and then transforms to tensor
#improve 하는법- 정규화

def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = [] #empty list

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        loop.set_postfix(loss=loss.item()) #proegress bar updates for every batch

    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}") #train entire dataset


def main():
    model = Yolov1(split_size=7, num_boxes=2, num_classes=20).to(DEVICE)
    optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
    loss_fn = YoloLoss()

    if LOAD_MODEL: 
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

    train_dataset = VOCDataset(
        "C:/Users/Owner/Downloads/archive/100examples.csv", #8examples?
        transform=transform,
        img_dir=IMG_DIR,
        label_dir=LABEL_DIR,
    )

    test_dataset = VOCDataset(
        "C:/Users/Owner/Downloads/archive/test.csv", transform=transform, img_dir=IMG_DIR, label_dir=LABEL_DIR,
    )

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True, #gradient update 안하기 위함, update하면 ruin gradients
    )

    for epoch in range(EPOCHS):
        # for x, y in train_loader:
        #    x = x.to(DEVICE)
        #    for idx in range(8):
        #        bboxes = cellboxes_to_boxes(model(x))
        #        bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
        #        plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes)

        #    import sys
        #    sys.exit()

        pred_boxes, target_boxes = get_bboxes(
            train_loader, model, iou_threshold=0.5, threshold=0.4 #non-max suppression
        )

        mean_avg_prec = mean_average_precision(
            pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint" 
        )
        print(f"Train mAP: {mean_avg_prec}")

        #if mean_avg_prec > 0.9:
        #    checkpoint = {
        #        "state_dict": model.state_dict(),
        #        "optimizer": optimizer.state_dict(),
        #    }
        #    save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)
        #    import time
        #    time.sleep(10)

        train_fn(train_loader, model, optimizer, loss_fn)


if __name__ == "__main__":
    main()

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [19]:
from loss import YoloLoss

seed = 123
torch.manual_seed(seed) #get same data set loading

# Hyperparameters etc. 
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 16 # 64 in original paper but I don't have that much vram, grad accum?
#batch size 64라고 언급된 논문 본적 없음, gpu 업그레이드 필요함
WEIGHT_DECAY = 0 #not train entire model- take a long time to train , 계산상의 이유로 무거운 데이터 증강, computational reasons
#정규화 없음
EPOCHS = 1000
NUM_WORKERS = 2
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar" #오버피팅하고 나중에 실행할 예정
IMG_DIR = "C:/Users/Owner/Downloads/archive/images"
LABEL_DIR = "C:/Users/Owner/Downloads/archive/labels"

In [20]:
#resize, transform.compose 는 bbx 포함 안할 수도
#resize 하면 조금 이상하게 보일 수도? not do any data augmentation on the label side
#데이터 증강이 많았으면 데이터 확장 쉬웠을것이다.
class Compose(object): 
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes = t(img), bboxes #t- transform, 이미지를 작동시키기 위함, 크기 조정만 할것임

        return img, bboxes


transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])

In [21]:
#resize and then transforms to tensor
#improve 하는법- 정규화

def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = [] #empty list

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        loop.set_postfix(loss=loss.item()) #proegress bar updates for every batch

    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}") #train entire dataset

In [22]:
def main():
    model = Yolov1(split_size=7, num_boxes=2, num_classes=20).to(DEVICE)
    optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
    loss_fn = YoloLoss()

    if LOAD_MODEL: 
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

    train_dataset = VOCDataset(
        "C:/Users/Owner/Downloads/archive/100examples.csv", #8examples?
        transform=transform,
        img_dir=IMG_DIR,
        label_dir=LABEL_DIR,
    )

    test_dataset = VOCDataset(
        "C:/Users/Owner/Downloads/archive/test.csv", transform=transform, img_dir=IMG_DIR, label_dir=LABEL_DIR,
    )

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True, #gradient update 안하기 위함, update하면 ruin gradients
    )

    for epoch in range(EPOCHS):
        # for x, y in train_loader:
        #    x = x.to(DEVICE)
        #    for idx in range(8):
        #        bboxes = cellboxes_to_boxes(model(x))
        #        bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
        #        plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes)

        #    import sys
        #    sys.exit()

        pred_boxes, target_boxes = get_bboxes(
            train_loader, model, iou_threshold=0.5, threshold=0.4 #non-max suppression
        )

        mean_avg_prec = mean_average_precision(
            pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint" 
        )
        print(f"Train mAP: {mean_avg_prec}")

        #if mean_avg_prec > 0.9:
        #    checkpoint = {
        #        "state_dict": model.state_dict(),
        #        "optimizer": optimizer.state_dict(),
        #    }
        #    save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)
        #    import time
        #    time.sleep(10)

        train_fn(train_loader, model, optimizer, loss_fn)


if __name__ == "__main__":
    main()

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx